Json Parsing

In [1]:
import copy
from pyspark.sql import SQLContext
import json

sqlContext = SQLContext(sc)
df = sqlContext.jsonFile("./spark_tutorial_article.json")

gf = df.map(lambda x : (x[2],x[5],x[12]))

print type(gf)
#spark.read.json(sc.wholeTextFiles('./spark_tutorial_article.json').values())

<class 'pyspark.rdd.PipelinedRDD'>


In [2]:
#sc.textFile("./spark_tutorial_article.json").map(json.loads).take(1)[0][u'author']

用BeautifulSoup擷取內容，並套用Jieba斷詞

In [3]:
## getContent: for input aritcle, get it own word set via jieba.cut()
def getContent(x):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(x)
    text = soup.getText().replace('\n','').replace('\r','').replace(' ','').replace('\t','')
    import jieba
    r = list()
    for term in jieba.cut(text):
        if len(term) > 1 and checkword(term): r.append(term)
    return r

def checkword(x):
    return all(u'\u4e00' <= c <= u'\u9fff' for c in x)

In [4]:
text_token = gf.map(lambda x: (x[0], getContent(x[1]), x[2]))

In [5]:
#check text_token
#text_token.first()
#text_token.count()

(u'\u7f8e\u5473\u98df\u8a18',
 [u'\u5bf6\u5bf6',
  u'\u5927\u4eba',
  u'\u4e00\u76f4',
  u'\u6536\u96c6',
  u'\u5403\u904d',
  u'\u53f0\u5317',
  u'\u9ad8\u7d1a',
  u'\u9910\u5ef3',
  u'\u98ef\u5e97',
  u'\u6676\u83ef\u827e\u7f8e',
  u'\u5bd2\u820d',
  u'\u7b49\u7b49',
  u'\u552f\u7368',
  u'\u559c\u4f86',
  u'\u5eda\u9084',
  u'\u6c92\u5403\u904e',
  u'\u79ae\u62dc\u524d',
  u'\u8a02\u4f4d',
  u'\u5176\u5be6',
  u'\u4e00\u76f4',
  u'\u5ba2\u6eff',
  u'\u53ea\u6709',
  u'\u500b\u79ae',
  u'\u62dc\u5f8c\u624d',
  u'\u4f4d\u7f6e',
  u'\u9019\u6b21',
  u'\u9084\u9080',
  u'\u4e09\u540d',
  u'\u53cb\u4eba',
  u'\u4e00\u540c',
  u'\u4eab\u53d7',
  u'\u9019\u4e00\u5ba2',
  u'\u9ad8\u7d1a',
  u'\u805a\u6703',
  u'\u9996\u5148',
  u'\u6211\u5011',
  u'\u873b\u8713',
  u'\u9ede\u6c34\u5f0f',
  u'\u7b56\u7565',
  u'\u5168\u5834',
  u'\u7d66\u5403\u904e',
  u'\u4e00\u904d',
  u'\u4f86\u91dd',
  u'\u597d\u5403',
  u'\u7e7c\u7e8c',
  u'\u8ffd\u52a0',
  u'\u4f46\u662f',
  u'\u5b8c\u4e00\u8f2a',
  u'

計算每篇文章的TF-IDF Vector

In [6]:
def cal_tf(tokens):
    d = {}
    for word in tokens:
        if not word in d:
            d[word] = 1
        else:
            d[word] += 1
    for word in d:
        d[word] = float(d[word])/len(tokens)
    return d

text_token_tf = text_token.map(lambda x: cal_tf(x[1]))

In [7]:
#check text_token_tf
#text_token_tf.first()

{u'\u4e00\u4eba': 0.003861003861003861,
 u'\u4e00\u500b': 0.003861003861003861,
 u'\u4e00\u540c': 0.003861003861003861,
 u'\u4e00\u6a23': 0.003861003861003861,
 u'\u4e00\u6bb5': 0.003861003861003861,
 u'\u4e00\u76f4': 0.007722007722007722,
 u'\u4e00\u7a2e': 0.003861003861003861,
 u'\u4e00\u904d': 0.003861003861003861,
 u'\u4e00\u9ede\u9ede': 0.003861003861003861,
 u'\u4e09\u540d': 0.003861003861003861,
 u'\u4e0b\u6b21': 0.003861003861003861,
 u'\u4e0d\u5230': 0.003861003861003861,
 u'\u4e0d\u6562': 0.003861003861003861,
 u'\u4e0d\u7528': 0.003861003861003861,
 u'\u4e0d\u89ba': 0.003861003861003861,
 u'\u4e1f\u4e09\u7c60': 0.003861003861003861,
 u'\u4e2d\u83ef': 0.003861003861003861,
 u'\u4e9b\u5c0f': 0.003861003861003861,
 u'\u4eab\u53d7': 0.003861003861003861,
 u'\u4ee5\u4e0a': 0.003861003861003861,
 u'\u4f46\u662f': 0.003861003861003861,
 u'\u4f4d\u7f6e': 0.003861003861003861,
 u'\u4f86\u91dd': 0.003861003861003861,
 u'\u500b\u79ae': 0.003861003861003861,
 u'\u507d\u99ac\u5361\u9f8d'

In [8]:
def cal_idf(docs):
    N = docs.count()
    uniqueTokens = docs.map(lambda x : list(set(x[1])))
    token_sum_tuples = uniqueTokens.flatMap(lambda x: x).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)
    return token_sum_tuples.map(lambda x : (x[0], float(N)/x[1]))

In [9]:
def TFIDF(tokens, idfs):
    tfs = cal_tf(tokens)
    for tk in tfs:
        tfs[tk] = tfs[tk]*idfs[tk]
    tfidf_Dict = tfs
    return tfidf_Dict

In [10]:
doc_idfs = cal_idf(text_token)

doc_c = doc_idfs.collectAsMap()  #my idf dict

text_tfidf =  TFIDF(text_token.collect()[0][1], doc_c)

print text_token.collect()[0][0]

<class 'pyspark.rdd.PipelinedRDD'>
<type 'dict'>
美味食記


In [11]:
#check text_tfidf
#text_tfidf
#text_token.collect()[0][1]

{u'\u4e00\u4eba': 0.04175881845784758,
 u'\u4e00\u500b': 0.008942117050225159,
 u'\u4e00\u540c': 0.09350344132952829,
 u'\u4e00\u6a23': 0.010944423158163616,
 u'\u4e00\u6bb5': 0.012994435955161032,
 u'\u4e00\u76f4': 0.028112145759204583,
 u'\u4e00\u7a2e': 0.025300931183284123,
 u'\u4e00\u904d': 0.6144511858797573,
 u'\u4e00\u9ede\u9ede': 0.056968984121302,
 u'\u4e09\u540d': 4.301158301158301,
 u'\u4e0b\u6b21': 0.032098196277300754,
 u'\u4e0d\u5230': 0.029561225437514098,
 u'\u4e0d\u6562': 0.04726547583690441,
 u'\u4e0d\u7528': 0.021559690732623066,
 u'\u4e0d\u89ba': 0.12839278510920302,
 u'\u4e1f\u4e09\u7c60': 8.602316602316602,
 u'\u4e2d\u83ef': 0.16542916542916544,
 u'\u4e9b\u5c0f': 0.4779064779064779,
 u'\u4eab\u53d7': 0.01874143050613639,
 u'\u4ee5\u4e0a': 0.02234367948653663,
 u'\u4f46\u662f': 0.011562253497737368,
 u'\u4f4d\u7f6e': 0.021240287906954574,
 u'\u4f86\u91dd': 8.602316602316602,
 u'\u500b\u79ae': 2.1505791505791505,
 u'\u507d\u99ac\u5361\u9f8d': 8.602316602316602,
 u'\

Cosine similarity

In [12]:
import math

def dotprod(a, b):
    dotsum = 0
    for tk in a:
        if tk in b:
            dotsum += a[tk]*b[tk]
    return dotsum

def norm(a):
    return math.sqrt(dotprod(a,a))

def cossim(a, b):
    return dotprod(a,b)/(norm(a) * norm(b))

In [13]:
def cosineSimilarity(string1, string2, idfsDictionary):
    w1 = tfidf(string1, idfsDictionary)
    w2 = tfidf(string2, idfsDictionary)
    return cossim(w1, w2)

Rule One - top words in a text

In [14]:
def showTopWord(link):
    tokens = text_token.filter(lambda x: x[2] == link).collect()[0][1]
    tokens_weights = TFIDF(tokens, doc_c)
    print type(tokens_weights)
    tokens_weights_sorted = sorted(tokens_weights, key=tokens_weights.get, reverse=True)
    for index in range(0,9):
        print tokens_weights_sorted[index], tokens_weights[tokens_weights_sorted[index]]
    print tokens_weights_sorted[:14]
    return tokens_weights_sorted[:14]

In [15]:
link = u'http://lovecc6.pixnet.net/blog/post/73513867'
showTopWord(link)

<type 'dict'>
上週 28.5641025641
為數 28.5641025641
半開始 6.34757834758
搬家 2.65712581992
摘要 1.73115773116
兩點 1.46482577252
請點 1.3765832561
全文 1.3601953602
地雷 1.14256410256
[u'\u4e0a\u9031', u'\u70ba\u6578', u'\u534a\u958b\u59cb', u'\u642c\u5bb6', u'\u6458\u8981', u'\u5169\u9ede', u'\u8acb\u9ede', u'\u5168\u6587', u'\u5730\u96f7', u'\u5de5\u5177', u'\u6280\u8853', u'\u7e7c\u7e8c', u'\u7a81\u7136', u'\u4e00\u5bb6']


[u'\u4e0a\u9031',
 u'\u70ba\u6578',
 u'\u534a\u958b\u59cb',
 u'\u642c\u5bb6',
 u'\u6458\u8981',
 u'\u5169\u9ede',
 u'\u8acb\u9ede',
 u'\u5168\u6587',
 u'\u5730\u96f7',
 u'\u5de5\u5177',
 u'\u6280\u8853',
 u'\u7e7c\u7e8c',
 u'\u7a81\u7136',
 u'\u4e00\u5bb6']

In [16]:
urls = text_token.map(lambda x : x[2])

#top_word_list = [showTopWord(i) for i in urls]
#top_word_list = urls.map(lambda x: showTopWord(x))
#top_word_list

Rule Two - Query in text

In [17]:
query_input = [u'淺草', u'雷門', u'和服']

def check_in(query, text):
    count = 0
    for q in query:
        if q in text:
            count += 1
    return count

def query_points(query):
    query_points_table = text_token.map(lambda x : check_in(query, x[1]))
    return query_points_table

In [73]:
query_pts = query_points(query_input).collect()

len(query_pts)

2228

Rule 3 - Term Weights

In [19]:
def term_weights(tokens):
    d = {}
    for word in tokens:
        if not word in d:
            d[word] = 1
        else:
            d[word] += 1
    return d

In [74]:
def term_points(query, point_dict):
    points = 0
    for i in query:
        if i in point_dict:
            points += point_dict[i]
                
    return points

tf_list = text_token.map(lambda x : term_weights(x[1])).collect()

In [75]:
term_pts = [term_points(query_input, i) for i in tf_list]
len(term_pts)

2228

In [22]:
def doc_points(term_weight_pts, query_pts):
#    tw_dict = text_token.map(lambda x: term_weights(x[1])).collect()
#    doc_point = text_token.map(lambda x : (((term_points(query_input, tw_dict))*(check_in(query_input, x[1])) , x[2])))
    doc_point = [i*j for i,j in zip(term_weight_pts, query_pts)]
    
    return doc_point

In [76]:
total_pts = doc_points(term_pts, query_pts)

In [77]:
print type(total_pts)
total_pts_sort = sorted(total_pts, reverse=True)
total_pts_sort

<type 'list'>


[8,
 3,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [78]:
url_list = text_token.map(lambda x : (x[2]))

In [79]:
url_list.collect()

[u'http://louis740321.pixnet.net/blog/post/373737533',
 u'http://jennifersec7.pixnet.net/blog/post/344940716',
 u'http://jill7708.pixnet.net/blog/post/236934860',
 u'http://castor0605.pixnet.net/blog/post/371068016',
 u'http://lovecc6.pixnet.net/blog/post/73513867',
 u'http://jende168.pixnet.net/blog/post/74962897',
 u'http://merecat.pixnet.net/blog/post/222227141-%e5%96%ab%ef%bc%8a%e5%8f%b0%e5%8c%97%ef%bc%8a%e5%96%9c%e4%be%86%e7%99%bb%e2%80%a7kitchen12%ef%bd%9e%e7%94%9c%e9%a3%9f%e7%9a%84%e5%a4%a9%e5%a0%82%e2%97%95%e2%80%bf',
 u'http://vilo92.pixnet.net/blog/post/434651582',
 u'http://abby0318.pixnet.net/blog/post/370358651',
 u'http://findse.pixnet.net/blog/post/426084965',
 u'http://reitin0110.pixnet.net/blog/post/108843454-%5b%e9%a3%9f%e4%ba%8b%5d%e5%8f%b0%e5%8c%97%e3%80%82%e5%96%9c%e4%be%86%e7%99%bb%e3%80%8212%e5%bb%9a',
 u'http://amy661204.pixnet.net/blog/post/117120718',
 u'http://lovedach.pixnet.net/blog/post/368240726',
 u'http://jill0426.pixnet.net/blog/post/365484134',
 u'htt

In [80]:
top_text = zip(total_pts_sort, url_list.collect())
print top_text
top_text.sort(key=lambda x: x[0], reverse=True)

[(8, u'http://louis740321.pixnet.net/blog/post/373737533'), (3, u'http://jennifersec7.pixnet.net/blog/post/344940716'), (2, u'http://jill7708.pixnet.net/blog/post/236934860'), (2, u'http://castor0605.pixnet.net/blog/post/371068016'), (2, u'http://lovecc6.pixnet.net/blog/post/73513867'), (2, u'http://jende168.pixnet.net/blog/post/74962897'), (2, u'http://merecat.pixnet.net/blog/post/222227141-%e5%96%ab%ef%bc%8a%e5%8f%b0%e5%8c%97%ef%bc%8a%e5%96%9c%e4%be%86%e7%99%bb%e2%80%a7kitchen12%ef%bd%9e%e7%94%9c%e9%a3%9f%e7%9a%84%e5%a4%a9%e5%a0%82%e2%97%95%e2%80%bf'), (1, u'http://vilo92.pixnet.net/blog/post/434651582'), (1, u'http://abby0318.pixnet.net/blog/post/370358651'), (1, u'http://findse.pixnet.net/blog/post/426084965'), (1, u'http://reitin0110.pixnet.net/blog/post/108843454-%5b%e9%a3%9f%e4%ba%8b%5d%e5%8f%b0%e5%8c%97%e3%80%82%e5%96%9c%e4%be%86%e7%99%bb%e3%80%8212%e5%bb%9a'), (1, u'http://amy661204.pixnet.net/blog/post/117120718'), (1, u'http://lovedach.pixnet.net/blog/post/368240726'), (1, u

TypeError: 'NoneType' object has no attribute '__getitem__'

In [82]:
top_text[:10]

[(8, u'http://louis740321.pixnet.net/blog/post/373737533'),
 (3, u'http://jennifersec7.pixnet.net/blog/post/344940716'),
 (2, u'http://jill7708.pixnet.net/blog/post/236934860'),
 (2, u'http://castor0605.pixnet.net/blog/post/371068016'),
 (2, u'http://lovecc6.pixnet.net/blog/post/73513867'),
 (2, u'http://jende168.pixnet.net/blog/post/74962897'),
 (2,
  u'http://merecat.pixnet.net/blog/post/222227141-%e5%96%ab%ef%bc%8a%e5%8f%b0%e5%8c%97%ef%bc%8a%e5%96%9c%e4%be%86%e7%99%bb%e2%80%a7kitchen12%ef%bd%9e%e7%94%9c%e9%a3%9f%e7%9a%84%e5%a4%a9%e5%a0%82%e2%97%95%e2%80%bf'),
 (1, u'http://vilo92.pixnet.net/blog/post/434651582'),
 (1, u'http://abby0318.pixnet.net/blog/post/370358651'),
 (1, u'http://findse.pixnet.net/blog/post/426084965')]